# Chains in LangChain

## Outline
* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [3]:
import datetime

current_date = datetime.datetime.now().date()

target_date = datetime.date(2024, 6, 12)

if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [4]:
# Import pandas
import pandas as pd

In [5]:
# Load dataset
df = pd.read_csv('l3_data.csv')
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [6]:
from langchain_openai import ChatOpenAI  # The OpenAI model
from langchain.prompts import ChatPromptTemplate  # The prompt
from langchain.chains import LLMChain  # The llm chain

In [7]:
# Initialize the language model
llm = ChatOpenAI(temperature=0.9)

In [8]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

In [9]:
# Combine the model and the prompt into a chain
chain = LLMChain(llm=llm, prompt=prompt)

In [10]:
product = "Queen size Sheet Set"
# chain.run(product)  # Deprecated use `invoke` instead of `run`
chain.invoke(product)

{'product': 'Queen size Sheet Set', 'text': '"Royal Slumber"'}

## SimpleSequentialChain

Sequential chains is another type of chains. The idea is to combine multiple chains where the output of the one chain is the input of the next chain.

There two type of sequential chains:
1. SimpleSequentialChain: Single input/outpur
2. SequentialChain: multiple inputs/outputs

Sequential chains run (invoke) a sequence of chains one after another.

In [11]:
# Import the simple sequential chain (one input/output)
from langchain.chains import SimpleSequentialChain

In [12]:
# Model
llm = ChatOpenAI(temperature=0.9)

# Prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}"
)

# Chain
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [13]:
# Prompt template 2
second_promt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following company: {company_name}"
)

# Chain 2
chain_two = LLMChain(llm=llm, prompt=second_promt)

In [14]:
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two],
    verbose=True
)

In [15]:
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
Regal Rest Bedding
Regal Rest Bedding offers luxurious and comfortable mattresses, pillows, and bedding essentials to help you get the best night's sleep.

> Finished chain.


{'input': 'Queen size Sheet Set',
 'output': "Regal Rest Bedding offers luxurious and comfortable mattresses, pillows, and bedding essentials to help you get the best night's sleep."}

## SequentialChain

Simple sequential chains work well when there is only a single input and a single output. But what when there are multiple inputs and multiple outputs?

In [16]:
from langchain.chains import SequentialChain

In [17]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [18]:
# Prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the followin review to english:"
    "\n\n{Review}"
)

# Chain 1: input = Review, output = English_Review
chain_one = LLMChain(
    llm=llm,
    prompt=first_prompt,
    output_key="English_Review"
)

In [19]:
# Prompt template 2: Summarize the english review
second_promt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)

# Chain 3: input = English_Review, output = summary
chain_two = LLMChain(
    llm=llm,
    prompt=second_promt,
    output_key="summary"
)

In [20]:
# Prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# Chain 3: input = Review, output = language
chain_three = LLMChain(
    llm=llm, prompt=third_prompt,
    output_key="language"
)


In [21]:

# Prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    """Write a follow up response to the following summary in the specified language:\
    \n\nSummary: {summary}\n\nLanguage: {language}"""
)
# Chain 4: input = summary, language and output = followup_message
chain_four = LLMChain(
    llm=llm, prompt=fourth_prompt,
    output_key="followup_message"
)


❗ For all the above subchains, the input key and the output key need to be precise.

In [22]:
# Overall chain: input = Review, output = English_Review, summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "followup_message"],
    verbose=True
)

In [23]:
review = df.Review[5]
# overall_chain(review)  # Deprecated, use invode instead 👇
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer found the taste of the product to be mediocre and suspects that they received an old batch or counterfeit item due to the foam not holding.',
 'followup_message': "Réponse: Nous sommes désolés d'apprendre que vous avez été déçu par le goût de notre produit. Nous nous assurons toujours de fournir des produits frais et authentiques à nos clients. Si vous pensez avoir reçu un produit non conforme, veuillez nous contacter pour que nous puissions enquêter sur ce qui s'est passé et trouver une solution. Nous apprécions vos commentaires et ferons tout notre possible pour améliorer notre produit."}

## RouterChain

In [24]:
physics_template = """You are a very smart physics professor. You are great at answering questions \
about physics in a concise and easy to understand manner. When you don't know the answer to a \
question you admit that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. You have an excellent knowledge of and \
understanding of people, events and contexts from a range of historical periods. You have the \
ability to think, reflect, debate, discuss and evaluate the past. You have a respect for historical \
evidence and the ability to make use of it to support your explanations and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist. You have a passion for \
creativity, collaboration, forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication skills. You are great at \
answering coding questions. You are so good because you know how to solve a problem by \
describing the solution in imperative steps that a machine can easily interpret and you know how to \
choose a solution that has a good balance between time complexity and space complexity. 

Here is a question:
{input}"""

In [25]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [26]:
# Import required type of chains
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [27]:
# Model
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [28]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [29]:
# Defautl chain
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [30]:
MULTI_PROMPT_ROUTER_TEMPLATE = """
Given a raw text input to a language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a description of what the prompt is \
best suited for. You may also revise the original input if you think that revising \
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can \
be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>
"""

In [31]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [32]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True
)

In [33]:
chain.invoke("The isolation politic for about two centurious in Japan")



> Entering new MultiPromptChain chain...
History: {'input': 'The isolation policy for about two centuries in Japan'}
> Finished chain.


{'input': 'The isolation policy for about two centuries in Japan',
 'text': "was a significant factor in shaping its unique culture and society. How did this policy come about, and what were its effects on Japan's development?"}

In [34]:
chain.invoke("How did the isolation policy come about, and what were its effects on Japan's development?")



> Entering new MultiPromptChain chain...
History: {'input': "How did the isolation policy come about, and what were its effects on Japan's development?"}
> Finished chain.


{'input': "How did the isolation policy come about, and what were its effects on Japan's development?",
 'text': "The isolation policy, also known as sakoku, was implemented by the Tokugawa shogunate in Japan in the 17th century. The policy aimed to limit foreign influence and maintain domestic stability by restricting contact with the outside world.\n\nThe isolation policy was a response to the increasing influence of European traders and missionaries in Japan, which threatened the traditional social and political order. The shogunate feared that foreign ideas and technologies would undermine their authority and lead to social unrest.\n\nUnder the isolation policy, Japan closed its ports to foreign ships and prohibited Japanese citizens from leaving the country. Only a limited number of Dutch and Chinese traders were allowed to trade with Japan through the port of Nagasaki.\n\nThe isolation policy had both positive and negative effects on Japan's development. On the one hand, it allow

In [35]:
chain.invoke("What is 2+2")



> Entering new MultiPromptChain chain...
math: {'input': 'What is 2+2'}
> Finished chain.


{'input': 'What is 2+2',
 'text': 'As an AI language model, I can answer this question. The answer to 2+2 is 4.'}

In [36]:
chain.invoke("What is black hole?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is a black hole?'}
> Finished chain.


{'input': 'What is a black hole?',
 'text': 'A black hole is a region in space where the gravitational pull is so strong that nothing, not even light, can escape it. It is formed when a massive star collapses in on itself, creating a singularity with infinite density and zero volume. The boundary around the black hole where the gravitational pull is so strong that nothing can escape is called the event horizon. Black holes are fascinating objects in the universe and have been the subject of much study and research in the field of astrophysics.'}

In [37]:
chain.invoke("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why is DNA important for the functioning of cells in our body?'}

ValueError: Received invalid destination chain name 'biology'

Here we are getting an error because `biology` does not appear to be one of the listed chains.